In [1]:
!pip install -q mlxtend


In [2]:
# === IMPORTS E CONFIGURAÇÕES ===
# Bibliotecas principais para manipulação de dados e análise
import pandas as pd
import numpy as np
from datetime import datetime

# Bibliotecas para visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Bibliotecas para machine learning
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Biblioteca para análise de market basket
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Biblioteca para salvar/carregar modelos
from joblib import dump, load

# Configurações de sistema e warnings
import os
import warnings
warnings.filterwarnings('ignore')

# Configuração de visualização
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)


# Navegar para o diretório de dados processados
try:
    os.chdir('../data/processed')
    print("Diretório configurado com sucesso!")
except Exception as e:
    print(f"Erro ao configurar diretório: {e}")

# === CARREGAMENTO E PREPARAÇÃO DOS DADOS ===

# 1. Carregar dados
try:
    # Carrega CSV com parsing de datas automático
    df = pd.read_csv('transacoes_final.csv', parse_dates=['DataFatura'])
    print(f"Dados carregados com sucesso! Total de registros: {len(df)}")
except Exception as e:
    print(f"Erro ao carregar dados: {e}")

# 2. Verificar e calcular valor total da fatura se necessário
if 'ValorTotalFatura' not in df.columns:
    df['ValorTotalFatura'] = df['Quantidade'] * df['PrecoUnitario']
    print("Coluna ValorTotalFatura criada!")

# 3. Preparação dos dados para análise RFM
# Converter e limpar IDs de cliente
df['IDCliente'] = df['IDCliente'].astype(str)

# Filtrar apenas clientes identificados
df_clientes = df[(df['IDCliente'].notna()) & (df['IDCliente'] != 'Desconhecido')]
print(f"Total de clientes identificados: {df_clientes['IDCliente'].nunique()}")

# 4. Definir data de referência para cálculo de recência
# Usamos o dia após a última compra como referência
data_referencia = df_clientes['DataFatura'].max() + pd.DateOffset(days=1)
print(f"Data de referência para análise: {data_referencia.strftime('%d/%m/%Y')}")

# 5. Criar dataframe RFM base
# Calcular métricas agregadas por cliente
rfm = df_clientes.groupby('IDCliente').agg({
    'DataFatura': lambda x: (data_referencia - x.max()).days,  # Recência
    'ValorTotalFatura': 'sum'  # Valor Monetário
}).reset_index()

# Renomear colunas para clareza
rfm.columns = ['IDCliente', 'Recencia', 'ValorMonetario']

# Calcular frequência separadamente (número único de faturas)
frequencia = df_clientes.groupby('IDCliente')['NumeroFatura'].nunique().reset_index()
frequencia.columns = ['IDCliente', 'Frequencia']

# Juntar todas as métricas RFM
rfm = rfm.merge(frequencia, on='IDCliente')

# 6. Limpeza e conversão de tipos
# Converter métricas para numérico e tratar valores ausentes
for coluna in ['Recencia', 'Frequencia', 'ValorMonetario']:
    rfm[coluna] = pd.to_numeric(rfm[coluna], errors='coerce')

# Remover registros com valores ausentes
rfm = rfm.dropna(subset=['Recencia', 'Frequencia', 'ValorMonetario'])
print(f"Dados RFM preparados com sucesso! Total de clientes: {len(rfm)}")

# 7. Validação dos dados
print("\nResumo das métricas RFM:")
print(rfm[['Recencia', 'Frequencia', 'ValorMonetario']].describe())

# === TRATAMENTO DE OUTLIERS E NORMALIZAÇÃO ===

# 1. Tratamento de Outliers
print("\nIniciando tratamento de outliers...")

# Armazenar limites originais para documentação
limites_originais = {}
for col in ['Recencia', 'Frequencia', 'ValorMonetario']:
    limites_originais[col] = {
        'min': rfm[col].min(),
        'max': rfm[col].max(),
        'mean': rfm[col].mean(),
        'std': rfm[col].std()
    }

# Aplicar clipping nos percentis 1 e 99
for col in ['Recencia', 'Frequencia', 'ValorMonetario']:
    # Calcular limites
    lower = rfm[col].quantile(0.01)
    upper = rfm[col].quantile(0.99)
    
    # Armazenar limites para documentação
    limites_tratados = {
        'lower': lower,
        'upper': upper
    }
    
    # Aplicar clipping
    rfm[col] = rfm[col].clip(lower=lower, upper=upper)
    
    print(f"\nTratamento de outliers para {col}:")
    print(f"Limite inferior: {lower:.2f}")
    print(f"Limite superior: {upper:.2f}")

# 2. Transformação logarítmica
print("\nAplicando transformação logarítmica...")
# Aplicar log1p (log(1+x)) para lidar com valores zero
rfm_log = rfm[['Recencia', 'Frequencia', 'ValorMonetario']].apply(np.log1p)

# 3. Normalização com StandardScaler
print("\nNormalizando dados...")
# Inicializar e ajustar o scaler
scaler = StandardScaler()
rfm_normalizado = scaler.fit_transform(rfm_log)

# Criar DataFrame normalizado para visualização
rfm_normalizado_df = pd.DataFrame(
    rfm_normalizado,
    columns=['Recencia', 'Frequencia', 'ValorMonetario'],
    index=rfm.index
)

# 4. Validação das transformações
print("\nValidação das transformações:")
print("Média próxima a 0 e desvio padrão próximo a 1 indicam normalização correta")
print(rfm_normalizado_df.describe())

# 5. Armazenar informações de pré-processamento
info_preprocessamento = {
    'limites_originais': limites_originais,
    'limites_tratados': limites_tratados,
    'transformacoes_aplicadas': ['log1p', 'StandardScaler'],
    'scaler_mean_': scaler.mean_.tolist(),
    'scaler_scale_': scaler.scale_.tolist()
}

print("\nPré-processamento concluído com sucesso!")

# === CLUSTERIZAÇÃO E ANÁLISE DOS SEGMENTOS ===

# 1. Aplicação do K-means
print("\nIniciando processo de clusterização...")
# Definir número de clusters
k = 4
kmeans = KMeans(
    n_clusters=k,
    random_state=42,
    n_init=10  # Número de vezes que o algoritmo será executado com diferentes centróides iniciais
)

# Treinar o modelo
kmeans.fit(rfm_normalizado)

# Adicionar labels ao DataFrame original
rfm['Cluster'] = kmeans.labels_

# 2. Avaliação do Modelo
print("\nAvaliando qualidade dos clusters...")
# Calcular pontuação de silhueta
silhouette_avg = silhouette_score(rfm_normalizado, kmeans.labels_)
print(f"Coeficiente de Silhueta: {silhouette_avg:.3f}")

# 3. Cálculo de Métricas Adicionais
print("\nCalculando métricas adicionais...")
# Adicionar métricas derivadas
rfm['UltimaCompra'] = data_referencia - pd.to_timedelta(rfm['Recencia'], unit='D')
rfm['CompraMediaPorVisita'] = rfm['ValorMonetario'] / rfm['Frequencia']
rfm['DiasEntreCompras'] = rfm['Recencia'] / rfm['Frequencia']

# 4. Análise dos Clusters
print("\nAnalisando características dos clusters...")
# Calcular métricas médias por cluster
cluster_metrics = rfm.groupby('Cluster')[
    ['Recencia', 'Frequencia', 'ValorMonetario', 'CompraMediaPorVisita', 'DiasEntreCompras']
].agg(['mean', 'count', 'std']).round(2)

# 5. Nomeação dos Clusters
# Definir nomes dos clusters baseados em múltiplas métricas
cluster_names = {
    0: 'Champions',
    1: 'Leais',
    2: 'Em Risco',
    3: 'Perdidos'
}

# Adicionar nomes dos segmentos ao DataFrame
rfm['Segmento'] = rfm['Cluster'].map(cluster_names)

# 6. Análise Detalhada dos Segmentos
print("\nAnálise detalhada por segmento:")
for cluster in sorted(cluster_names.keys()):
    segment_data = rfm[rfm['Cluster'] == cluster]
    nome_segmento = cluster_names[cluster]
    
    print(f"\nSegmento: {nome_segmento}")
    print(f"Quantidade de clientes: {len(segment_data)} ({len(segment_data)/len(rfm)*100:.1f}%)")
    print("\nMétricas médias:")
    print(f"- Recência: {segment_data['Recencia'].mean():.1f} dias")
    print(f"- Frequência: {segment_data['Frequencia'].mean():.1f} compras")
    print(f"- Valor médio: R$ {segment_data['ValorMonetario'].mean():,.2f}")
    print(f"- Ticket médio: R$ {segment_data['CompraMediaPorVisita'].mean():,.2f}")

# 7. Integrar resultados aos dados originais
df = df.merge(rfm[['IDCliente', 'Cluster', 'Segmento']], on='IDCliente', how='left')

# 8. Armazenar métricas do modelo
metricas_modelo = {
    'silhouette_score': silhouette_avg,
    'inertia': kmeans.inertia_,
    'distribuicao_clusters': rfm['Cluster'].value_counts().to_dict(),
    'metricas_por_cluster': cluster_metrics.to_dict()
}

print("\nClusterização e análise concluídas com sucesso!")

# === ANÁLISE DE MARKET BASKET POR SEGMENTO ===

print("\nIniciando análise de Market Basket por segmento...")

# 1. Preparação para análise de Market Basket
regras_clusters = []  # Lista para armazenar regras de cada cluster
metricas_market_basket = {}  # Dicionário para armazenar métricas

# 2. Análise por cluster
for cluster in df['Cluster'].dropna().unique():
    print(f"\nProcessando Cluster {cluster} ({cluster_names[cluster]})...")
    
    # Filtrar dados do cluster
    df_cluster = df[df['Cluster'] == cluster]
    df_cluster = df_cluster.dropna(subset=['NumeroFatura', 'Descricao'])
    
    # Criar identificador único de transação
    df_cluster['TransacaoID'] = df_cluster['NumeroFatura'].astype(str) + '_' + df_cluster['IDCliente'].astype(str)
    
    # Criar matriz de cestas de compras
    print(f"Criando matriz de cestas para o segmento {cluster_names[cluster]}...")
    basket_cluster = df_cluster.groupby(['TransacaoID', 'Descricao'])['Quantidade'].sum().unstack().fillna(0)
    
    # Converter para matriz binária (0 ou 1)
    basket_cluster = basket_cluster.applymap(lambda x: 1 if x >= 1 else 0)
    
    # Armazenar métricas iniciais
    metricas_market_basket[cluster] = {
        'total_transacoes': len(basket_cluster),
        'total_produtos': len(basket_cluster.columns)
    }
    
    # Filtrar produtos frequentes (mínimo de 20 ocorrências)
    item_counts = basket_cluster.sum(axis=0)
    basket_cluster = basket_cluster.loc[:, item_counts >= 20]
    
    print(f"Produtos após filtragem: {len(basket_cluster.columns)}")
    
    # 3. Geração de regras de associação
    if len(basket_cluster.columns) > 1:
        try:
            # Encontrar conjuntos frequentes
            frequent_itemsets = fpgrowth(
                basket_cluster,
                min_support=0.01,  # Suporte mínimo de 1%
                use_colnames=True
            )
            
            # Gerar regras de associação
            regras = association_rules(
                frequent_itemsets,
                metric='confidence',
                min_threshold=0.1  # Confiança mínima de 10%
            )
            
            # Adicionar informações do cluster
            regras['Cluster'] = cluster
            regras['Segmento'] = cluster_names[cluster]
            
            # Armazenar métricas
            metricas_market_basket[cluster].update({
                'total_itemsets': len(frequent_itemsets),
                'total_regras': len(regras),
                'confidence_media': regras['confidence'].mean(),
                'lift_medio': regras['lift'].mean()
            })
            
            # Adicionar à lista de regras
            regras_clusters.append(regras)
            
            print(f"Regras geradas para {cluster_names[cluster]}: {len(regras)}")
            
        except Exception as e:
            print(f"Erro na geração de regras para cluster {cluster}: {e}")
    else:
        print(f"Produtos insuficientes para análise no cluster {cluster}")

# 4. Consolidar todas as regras
if regras_clusters:
    regras_todas = pd.concat(regras_clusters, ignore_index=True)
    print(f"\nTotal de regras geradas: {len(regras_todas)}")
    
    # 5. Análise das melhores regras por segmento
    print("\nMelhores regras por segmento:")
    for segmento in regras_todas['Segmento'].unique():
        regras_segmento = regras_todas[regras_todas['Segmento'] == segmento]
        print(f"\nTop 3 regras para {segmento}:")
        top_regras = regras_segmento.nlargest(3, 'lift')
        for _, regra in top_regras.iterrows():
            print(f"Se compra {list(regra['antecedents'])}")
            print(f"Também compra {list(regra['consequents'])}")
            print(f"Confiança: {regra['confidence']:.2%}")
            print(f"Lift: {regra['lift']:.2f}\n")

else:
    print("Nenhuma regra de associação foi gerada")

print("\nAnálise de Market Basket concluída!")

# === GERAÇÃO DE INSIGHTS E SALVAMENTO DO MODELO ===

print("\nIniciando geração de insights e salvamento do modelo...")

# 1. Preparar diretórios
try:
    # Voltar para o diretório principal e criar pastas necessárias
    os.chdir('../../')
    os.makedirs('models', exist_ok=True)
    os.makedirs('documents', exist_ok=True)
    print("Diretórios criados/verificados com sucesso!")
except Exception as e:
    print(f"Erro ao criar diretórios: {e}")

# 2. Gerar relatório de insights
print("\nGerando relatório de insights...")
with open('documents/analise_detalhada_clientes.txt', 'w', encoding='utf-8') as f:
    # Cabeçalho
    f.write("=== ANÁLISE DETALHADA DE SEGMENTAÇÃO DE CLIENTES E PADRÕES DE COMPRA ===\n\n")
    
    # Seção 1: Visão Geral
    f.write("1. VISÃO GERAL DA BASE DE CLIENTES\n")
    f.write("-" * 50 + "\n")
    f.write(f"Total de clientes analisados: {len(rfm)}\n")
    f.write(f"Período de análise: {df['DataFatura'].min().strftime('%d/%m/%Y')} a {df['DataFatura'].max().strftime('%d/%m/%Y')}\n")
    f.write(f"Valor total transacionado: R$ {df['ValorTotalFatura'].sum():,.2f}\n\n")

    # Seção 2: Análise por Segmento
    f.write("2. ANÁLISE DETALHADA DOS SEGMENTOS\n")
    f.write("-" * 50 + "\n")
    
    for cluster in sorted(cluster_names.keys()):
        segment_data = rfm[rfm['Cluster'] == cluster]
        nome_segmento = cluster_names[cluster]
        
        f.write(f"\nSegmento: {nome_segmento}\n")
        f.write(f"Quantidade de clientes: {len(segment_data)} ({len(segment_data)/len(rfm)*100:.1f}%)\n")
        
        # Características do segmento
        f.write("\nCaracterísticas principais:\n")
        f.write(f"- Recência média: {segment_data['Recencia'].mean():.1f} dias\n")
        f.write(f"- Frequência média: {segment_data['Frequencia'].mean():.1f} compras\n")
        f.write(f"- Valor médio gasto: R$ {segment_data['ValorMonetario'].mean():,.2f}\n")
        f.write(f"- Valor médio por compra: R$ {segment_data['CompraMediaPorVisita'].mean():,.2f}\n")
        f.write(f"- Média de dias entre compras: {segment_data['DiasEntreCompras'].mean():.1f}\n")
        
        # Recomendações específicas por segmento
        f.write("\nInsights e Recomendações:\n")
        if nome_segmento == 'Champions':
            f.write("- Clientes de alto valor que compram frequentemente e recentemente\n")
            f.write("- Foco em retenção e programas de fidelidade premium\n")
            f.write("- Oportunidade para desenvolvimento de produtos premium\n")
            f.write("- Potenciais embaixadores da marca\n")
        elif nome_segmento == 'Leais':
            f.write("- Base sólida de clientes regulares\n")
            f.write("- Potencial para aumentar frequência de compras\n")
            f.write("- Implementar programa de recompensas por frequência\n")
            f.write("- Foco em cross-selling de produtos complementares\n")
        elif nome_segmento == 'Em Risco':
            f.write("- Sinais de diminuição na frequência de compras\n")
            f.write("- Necessidade de ação rápida para reengajamento\n")
            f.write("- Pesquisar motivos de afastamento\n")
            f.write("- Ofertas personalizadas baseadas em histórico de compras\n")
        else:  # Perdidos
            f.write("- Alta probabilidade de churn\n")
            f.write("- Campanha de reativação com ofertas agressivas\n")
            f.write("- Análise de causas de abandono\n")
            f.write("- Considerar remarketing específico\n")

    # Seção 3: Análise de Market Basket
    f.write("\n3. PADRÕES DE COMPRA POR SEGMENTO\n")
    f.write("-" * 50 + "\n")
    
    if 'regras_todas' in locals():
        for segmento in regras_todas['Segmento'].unique():
            f.write(f"\nPadrões de Compra - Segmento {segmento}\n")
            regras_segmento = regras_todas[regras_todas['Segmento'] == segmento].sort_values('lift', ascending=False).head(5)
            
            if not regras_segmento.empty:
                f.write("\nPrincipais associações de produtos:\n")
                for _, regra in regras_segmento.iterrows():
                    antecedents = ', '.join(list(regra['antecedents']))
                    consequents = ', '.join(list(regra['consequents']))
                    f.write(f"\n- Quando compram: {antecedents}")
                    f.write(f"\n  Também tendem a comprar: {consequents}")
                    f.write(f"\n  Confiança: {regra['confidence']:.2%}")
                    f.write(f"\n  Lift: {regra['lift']:.2f}\n")

    # Seção 4: Recomendações Estratégicas
    f.write("\n4. RECOMENDAÇÕES ESTRATÉGICAS GERAIS\n")
    f.write("-" * 50 + "\n")
    f.write("\nCurto Prazo (1-3 meses):\n")
    f.write("- Implementar campanhas de reativação para segmento 'Perdidos'\n")
    f.write("- Desenvolver programa de fidelidade segmentado\n")
    f.write("- Ajustar mix de produtos baseado nas análises de market basket\n")
    
    f.write("\nMédio Prazo (3-6 meses):\n")
    f.write("- Estabelecer metas de migração entre segmentos\n")
    f.write("- Desenvolver estratégias de up-selling para segmento 'Leais'\n")
    f.write("- Implementar sistema de alertas para clientes em risco de churn\n")
    
    f.write("\nLongo Prazo (6-12 meses):\n")
    f.write("- Revisar e ajustar segmentação baseado em resultados\n")
    f.write("- Desenvolver produtos/serviços específicos por segmento\n")
    f.write("- Estabelecer programa de customer success para Champions\n")

print("Relatório de insights gerado com sucesso!")

# === SALVAMENTO DO MODELO E METADATA ===

print("\nIniciando processo de salvamento do modelo e metadata...")

# 1. Preparar informações do modelo
model_info = {
    # Configurações básicas do modelo
    'n_clusters': kmeans.n_clusters,
    'feature_names': ['Recencia', 'Frequencia', 'ValorMonetario'],
    'data_treinamento': str(datetime.now()),
    
    # Informações sobre o pré-processamento
    'pre_processamento': {
        'outliers_tratamento': {
            'metodo': 'clipping',
            'percentis': {'inferior': 0.01, 'superior': 0.99}
        },
        'transformacoes': ['log1p', 'StandardScaler'],
        'scaler_info': {
            'mean': scaler.mean_.tolist(),
            'scale': scaler.scale_.tolist()
        }
    },
    
    # Métricas de avaliação
    'metricas': {
        'inertia': float(kmeans.inertia_),
        'silhouette': float(silhouette_avg),
        'tamanho_clusters': rfm['Cluster'].value_counts().to_dict(),
        'metricas_por_cluster': cluster_metrics.to_dict()
    },
    
    # Parâmetros do modelo
    'parametros': {
        'n_init': kmeans.n_init,
        'random_state': 42,
        'algorithm': kmeans.algorithm
    },
    
    # Mapeamento dos segmentos
    'segmentos': {
        'Champions': 'Cluster 0: Clientes de alto valor, frequentes e recentes',
        'Leais': 'Cluster 1: Base sólida de clientes regulares',
        'Em Risco': 'Cluster 2: Clientes com sinais de afastamento',
        'Perdidos': 'Cluster 3: Alta probabilidade de churn'
    },
    
    # Informações sobre os dados
    'dados': {
        'total_clientes': len(rfm),
        'periodo_analise': {
            'inicio': df['DataFatura'].min().strftime('%Y-%m-%d'),
            'fim': df['DataFatura'].max().strftime('%Y-%m-%d')
        }
    },
    
    # Informações do Market Basket
    'market_basket': metricas_market_basket
}

# 2. Salvar modelo e componentes
try:
    # Salvar o modelo K-means
    dump(kmeans, 'models/customer_segments.joblib')
    print("Modelo K-means salvo com sucesso!")
    
    # Salvar o scaler
    dump(scaler, 'models/scaler.joblib')
    print("Scaler salvo com sucesso!")
    
    # Salvar metadata do modelo
    dump(model_info, 'models/model_info.joblib')
    print("Metadata do modelo salva com sucesso!")
    
except Exception as e:
    print(f"Erro ao salvar arquivos: {e}")

# 3. Teste de carregamento
print("\nRealizando testes de carregamento...")
try:
    # Carregar e testar modelo
    modelo_teste = load('models/customer_segments.joblib')
    scaler_teste = load('models/scaler.joblib')
    info_teste = load('models/model_info.joblib')
    
    print("Testes de carregamento realizados com sucesso!")
    print(f"Número de clusters do modelo carregado: {modelo_teste.n_clusters}")
    
except Exception as e:
    print(f"Erro nos testes de carregamento: {e}")

# 4. Resumo final
print("\n=== RESUMO DA EXECUÇÃO ===")
print(f"Total de registros processados: {len(df)}")
print(f"Total de clientes segmentados: {len(rfm)}")
print(f"Arquivo de insights gerado: documents/analise_detalhada_clientes.txt")
print(f"Modelos e metadata salvos em: /models/")
print("\nProcesso concluído com sucesso!")

Diretório configurado com sucesso!
Dados carregados com sucesso! Total de registros: 460163
Total de clientes identificados: 4196
Data de referência para análise: 10/12/2011
Dados RFM preparados com sucesso! Total de clientes: 4196

Resumo das métricas RFM:
          Recencia   Frequencia  ValorMonetario
count  4196.000000  4196.000000     4196.000000
mean     92.400143     4.031935     1081.733072
std      99.874731     7.060200     2337.948654
min       1.000000     1.000000        1.900000
25%      18.000000     1.000000      218.070000
50%      51.000000     2.000000      497.290000
75%     144.000000     4.000000     1193.500000
max     374.000000   198.000000    91204.820000

Iniciando tratamento de outliers...

Tratamento de outliers para Recencia:
Limite inferior: 1.00
Limite superior: 369.00

Tratamento de outliers para Frequencia:
Limite inferior: 1.00
Limite superior: 26.00

Tratamento de outliers para ValorMonetario:
Limite inferior: 20.80
Limite superior: 7945.49

Aplicand